## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [42]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader #popular PDF library
import os
import gradio as gr # for generationg UI design

In [43]:
load_dotenv(override=True)
#groq_base_url = "https://api.groq.com/openai/v1"
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=openai_api_key)

In [44]:
reader = PdfReader("me/linkedinProfile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [45]:
print(linkedin)

   
Contact
8553467042 (Mobile)
deva9709@gmail.com
www.linkedin.com/in/dev-
sahu-615b1794 (LinkedIn)
Top Skills
Computer Science
GitHub Copilot
Azure Cosmos DB
Languages
English
Kannada (Elementary)
Hindi
Certifications
Using Docker and .NET Core
Microsoft Certified: Azure
Fundamentals
GITHub-CoPilot
Discovering .NET Aspire
Publications
devasahu.wordpress.com
Dev Sahu
Senior Software Engineer at Wipro
Bangalore Urban, Karnataka, India
Summary
Handling .net modules on C# with Asp.net core and angualr, SQL
Server technologies, identify the critical issues and implementing the
new requirements as per user need. Have been exposure on cloud
app services and deployment.
Experience
Wipro
Senior Software Engineer
October 2021 - Present (4 years 3 months)
Bangalore Urban, Karnataka, India
Working with great team and client to meet the mutual expectations. Be a part
of agile team who can deliver with top nitch product delivery
IIHT Ltd
Software Developer
September 2020 - July 2021 (11 months)
Ba

In [46]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [47]:
name = "Dev Sahu"

In [48]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [49]:
system_prompt

"You are acting as Dev Sahu. You are answering questions on Dev Sahu's website, particularly questions related to Dev Sahu's career, background, skills and experience. Your responsibility is to represent Dev Sahu for interactions on the website as faithfully as possible. You are given a summary of Dev Sahu's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHey, I’m DEV Sahu — a Full Stack developer at wipro who loves building Azure and Web applicaiton that actually work — scalable, reliable, and smart enough to keep learning.\nI love learning new skills and my future pathway to gain some knowledge about AI which includes GEN AI and Agentic AI.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n8553467042 (Mobile)\ndeva9709@gmail.com\nwww.linkedin.com/in/dev-\nsahu-615b1794 (LinkedIn)\nTop Skills\nComputer

In [80]:
model_name = "gpt-4o-mini"
ollama_model = "llama3.2"

In [57]:
def chat(message, history):
    # Clean history to only include role + content
    #history = [{"role": h["role"], "content": h["content"]} for h in history if "role" in h and "content" in h]

    # Build full message list
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    # Call Groq API
    response = openai.chat.completions.create(model=model_name, messages=messages)

    # Return assistant reply
    return response.choices[0].message.content
     

#Builds a conversation context (system prompt + history + new user message).
#Sends it to Groq’s chat completion API.
#Returns the model’s reply as plain text.
#It’s essentially a chat loop helper — you can call chat("Hello", history) and get back the AI’s response, while maintaining context across turns.


## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [58]:
 
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [59]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [60]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [63]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt
    #print(user_prompt)

In [81]:
import os
ollama = OpenAI(
    api_key=os.getenv("ollama"), 
    base_url="http://localhost:11434/v1"
)

In [82]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model=ollama_model, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [77]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you any certification?"}]
response = openai.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [78]:
reply

'Yes, I hold several certifications that reflect my skills and expertise in the field. Here are the key certifications I possess:\n\n1. **Using Docker and .NET Core**\n2. **Microsoft Certified: Azure Fundamentals**\n3. **GitHub Copilot Certification**\n4. **Discovering .NET Aspire**\n\nThese certifications help reinforce my knowledge and practical skills in areas like cloud computing and software development. If you have any specific questions about these certifications or how they relate to my work, feel free to ask!'

In [84]:
evaluate(reply, "do you hold a certifications?", messages[:1])

Evaluation(is_acceptable=True, feedback="The Agent's response is generally good, but it could be improved by adding a bit more context about the specific certifications they hold in relation to their work experience. For instance, explaining how using Docker and .NET Core has helped with cloud computing projects or highlighting how their Microsoft Certified: Azure Fundamentals certification assisted in managing Azure applications. The mention of GitHub Copilot Certification and Discovering .NET Aspire is also good, but it's worth noting that not all certifications are created equal in terms of relevance to software development expertise as a full stack developer.")

In [98]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [99]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model_name, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [100]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
